This is a brief stub of a notebook showing the use of SymPy in manipulating a more complicated form of the Coulomb interaction kernel for a more complicated semiempirical model than what you are using in your QM project.

In [1]:
import numpy as np
import sympy

As is typically the case in quantum chemistry, we will treat the atomi nuclei as classical point charges,

$$ \vec{r}_i = (x_i , y_i, z_i). $$

While we would normally read these from a file or pass them in as an argument to a function, we will just specify a set of atomic coordinates in this example:

In [2]:
atomic_coordinates = np.array([ [0.0,0.0,0.0], [3.0,4.0,5.0] ])
number_of_atoms = len(atomic_coordinates)

orbital_types = ['s', 'px' ,'py', 'pz']
orbitals_per_atom = len(orbital_types)

p_orbitals = orbital_types[1:]
print(p_orbitals)

np.set_printoptions(precision=1)
print(number_of_atoms)
print(atomic_coordinates)

def atom(ao_index):
    '''Returns the atom index part of an atomic orbital index.'''
    return ao_index // orbitals_per_atom

def orb(ao_index):
    '''Returns the orbital type of an atomic orbital index.'''
    orb_index = ao_index % orbitals_per_atom
    return orbital_types[orb_index]

def ao_index(atom_p,orb_p):
    '''Returns the atomic orbital index for a given atom index and orbital type.'''
    p = atom_p*orbitals_per_atom
    p += orbital_types.index(orb_p)
    return p

['px', 'py', 'pz']
2
[[0. 0. 0.]
 [3. 4. 5.]]


The nuclear charge of Argon is 18, but our model combines the nucleus and the 12 neglected electrons ($1s^2$, $2s^2$, $2p^6$, and $3s^2$) as an ionic point charge with $Z = 6$. For the purpose of electrostatics, we will describe the valence electrons of each Argon atom as Gaussian charges and their derivatives, which form a multipole expansion. The width of these Gaussians, $r_0$, is the first parameter of our semiempirical model:

In [3]:
ionic_charge = 6.0
r0 = 6.5

The electron-ion and electron-electron interactions have the same functional form, but the electron-electron interaction has an wider effective width of $\sqrt{2}r_0$,

$$ V^{\mathrm{II}}(r) = \frac{Z^2}{r} \\
   V^{\mathrm{eI}}(r) = -Z \frac{\mathrm{erf}(r/r_0)}{r} \\
   V^{\mathrm{ee}}(r) = \frac{\mathrm{erf}(r/(\sqrt{2}r_0))}{r}. $$

These functions can be constructed using SymPy, which will allow us to evaluate analytical derivatives automatically:

In [4]:
x1, y1, z1, x2, y2, z2, r = sympy.symbols('x1 y1 z1 x2 y2 z2 r')
r12 = sympy.sqrt( (x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2 )

erf_kernel = sympy.erf(r) / r
erf_kernel_close = erf_kernel.series(r,0,3).removeO()

v_ii_kernel = ionic_charge**2 / r12
v_ei_kernel = -ionic_charge*erf_kernel.subs(r, r12/r0) / r0
v_ee_kernel = erf_kernel.subs(r, r12/(np.sqrt(2.0)*r0)) / (np.sqrt(2.0)*r0)

v_ei_kernel_close = -ionic_charge*erf_kernel_close.subs(r, r12/r0) / r0
v_ee_kernel_close = erf_kernel_close.subs(r, r12/(np.sqrt(2.0)*r0)) / (np.sqrt(2.0)*r0)

def eval_kernel(kernel):
    '''Returns a NumPy function that efficiently evaluates the SymPy function input.'''
    return sympy.lambdify(([x1,y1,z1], [x2,y2,z2]), kernel)

print(erf_kernel)
print(erf_kernel_close)

erf(r)/r
-2*r**2/(3*sqrt(pi)) + 2/sqrt(pi)


SymPy is not smart enough to take limits automatically when they are needed, so we need to do this ourselves to avoid numerical singularities. It is then straightforward (but slow) to "compile" a function from SymPy to NumPy to evaluate formulas. For example, the ion-ion energy in $\hat{H}$,

$$ E_{\mathrm{ion}} = \sum_{i < j} V^{\mathrm{II}}(|\vec{r}_i - \vec{r}_j|) $$

can be implemented as:

In [5]:
def calculate_energy_ion(atomic_coordinates):
    '''Returns the ionic contribution to the total energy for an input list of atomic coordinates.'''
    energy_ion = 0.0
    v_ii = eval_kernel( v_ii_kernel )
    for i, r_i in enumerate(atomic_coordinates):
        for j, r_j in enumerate(atomic_coordinates):
            if i < j:
                energy_ion += v_ii(r_i, r_j)
    return energy_ion

energy_ion = calculate_energy_ion(atomic_coordinates)
print(energy_ion)

5.091168824543142


For functions involving electrons, we need to define a differential operator that will generate our multipole expansions,

$$ \Delta_p = \begin{cases} 1 , & \mathrm{orb}(p) = s \\
d/d x_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_x \\
d/d y_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_y \\
d/d z_{\mathrm{atom}(p)} , & \mathrm{orb}(p) = p_z
\end{cases}, $$

In [6]:
def grad1(kernel,orb1):
    '''Returns the input interaction kernel with a multipole generator in the 1st coordinate applied to it.'''
    if orb1 == 's':
        return kernel
    if orb1 == 'px':
        return sympy.diff(kernel, x1)
    if orb1 == 'py':
        return sympy.diff(kernel, y1)
    if orb1 == 'pz':
        return sympy.diff(kernel, z1)

def grad2(kernel,orb2):
    '''Returns the input interaction kernel with a multipole generator in the 2nd coordinate applied to it.'''
    if orb2 == 's':
        return kernel
    if orb2 == 'px':
        return sympy.diff(kernel, x2)
    if orb2 == 'py':
        return sympy.diff(kernel, y2)
    if orb2 == 'pz':
        return sympy.diff(kernel, z2)

print("function:\n", v_ee_kernel)
print("x1 derivative:\n", grad1(v_ee_kernel,'px'))
print("x1 & x2 derivatives:\n", grad2(grad1(v_ee_kernel,'px'),'px'))

function:
 1.0*erf(0.108785658644084*sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2))/sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
x1 derivative:
 1.0*(-x1 + x2)*erf(0.108785658644084*sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2))/((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)**(3/2) + 0.217571317288168*(x1 - x2)*exp(-0.0118343195266272*(x1 - x2)**2 - 0.0118343195266272*(y1 - y2)**2 - 0.0118343195266272*(z1 - z2)**2)/(sqrt(pi)*((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2))
x1 & x2 derivatives:
 0.217571317288168*(-x1 + x2)**2*exp(-0.0118343195266272*(x1 - x2)**2 - 0.0118343195266272*(y1 - y2)**2 - 0.0118343195266272*(z1 - z2)**2)/(sqrt(pi)*((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)**2) + 1.0*(-x1 + x2)*(3*x1 - 3*x2)*erf(0.108785658644084*sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2))/((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)**(5/2) + 0.217571317288168*(0.0236686390532544*x1 - 0.0236686390532544*x2)*(x1 - x2)*exp(-0.0118343195266272*(x1 - x2)**2 - 0.0118343195266272*(y1 - y

both for the vector of electron-ion potential matrix elements,

$$ V^{\mathrm{eI}}_p = \sum_{i} \Delta_p V^{\mathrm{eI}}(|\vec{r}_{\mathrm{atom}(p)} - \vec{r}_i|) , $$

In [7]:
def calculate_potential_vector(atomic_coordinates):
    '''Returns the electron-ion potential energy vector for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    potential_vector = np.zeros(ndof)
    for orb_p in orbital_types:
        v_ei = eval_kernel( grad1(v_ei_kernel, orb_p) )
        v_ei_close = eval_kernel( grad1(v_ei_kernel_close, orb_p) )
        for atom_p,r_p in enumerate(atomic_coordinates):
            p = ao_index(atom_p, orb_p)
            for atom_i,r_i in enumerate(atomic_coordinates):
                if atom_i != atom_p:
                    potential_vector[p] += v_ei(r_p, r_i)
                else:
                    potential_vector[p] += v_ei_close(r_p,r_i)
    return potential_vector

potential_vector = calculate_potential_vector(atomic_coordinates)
print(potential_vector)

[-1.8 -0.  -0.  -0.  -1.8  0.   0.   0. ]


and the matrix of electron-electron interaction matrix elements,

$$ V^{\mathrm{ee}}_{p,q} = \Delta_p \Delta_q V^{\mathrm{ee}}(|\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)}|) . $$

In [8]:
def calculate_interaction_matrix(atomic_coordinates):
    '''Returns the electron-electron interaction energy matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    interaction_matrix = np.zeros( (ndof,ndof) )
    for orb_p in orbital_types:
        for orb_q in orbital_types:
            v_ee = eval_kernel( grad2(grad1(v_ee_kernel, orb_p), orb_q) )
            v_ee_close = eval_kernel( grad2(grad1(v_ee_kernel_close, orb_p), orb_q) )
            for atom_p,r_p in enumerate(atomic_coordinates):
                p = ao_index(atom_p, orb_p)
                for atom_q,r_q in enumerate(atomic_coordinates):
                    q = ao_index(atom_q, orb_q)
                    if atom_p != atom_q:
                        interaction_matrix[p,q] = v_ee(r_p, r_q)
                    else:
                        interaction_matrix[p,q] = v_ee_close(r_p, r_q)
    return interaction_matrix

interaction_matrix = calculate_interaction_matrix(atomic_coordinates)
print(interaction_matrix)

[[ 1.2e-01 -0.0e+00 -0.0e+00 -0.0e+00  1.0e-01 -2.1e-03 -2.7e-03 -3.4e-03]
 [-0.0e+00  9.7e-04  0.0e+00  0.0e+00  2.1e-03  6.1e-04 -1.1e-04 -1.4e-04]
 [-0.0e+00  0.0e+00  9.7e-04  0.0e+00  2.7e-03 -1.1e-04  5.4e-04 -1.8e-04]
 [-0.0e+00  0.0e+00  0.0e+00  9.7e-04  3.4e-03 -1.4e-04 -1.8e-04  4.6e-04]
 [ 1.0e-01  2.1e-03  2.7e-03  3.4e-03  1.2e-01 -0.0e+00 -0.0e+00 -0.0e+00]
 [-2.1e-03  6.1e-04 -1.1e-04 -1.4e-04 -0.0e+00  9.7e-04  0.0e+00  0.0e+00]
 [-2.7e-03 -1.1e-04  5.4e-04 -1.8e-04 -0.0e+00  0.0e+00  9.7e-04  0.0e+00]
 [-3.4e-03 -1.4e-04 -1.8e-04  4.6e-04 -0.0e+00  0.0e+00  0.0e+00  9.7e-04]]
